In [3]:
# 📒 hackathon_itinerary_pipeline.ipynb

# ===============================
# 1️⃣ Setup & Imports
# ===============================
import sys, os


from features.iternary_generation.iternary_generator import generate_itinerary
from features.reddit_scraper.scraper import fetch_reddit_comments
from features.predictive_pipeline.local_intelligence import extract_local_tips
from features.predictive_pipeline.crowd_optimizer import optimize_crowd
from features.predictive_pipeline.weather_optimizer import optimize_weather
from features.predictive_pipeline.humanise_iternary import narrative_itinerary
from features.predictive_pipeline.place_reviews_builder import build_place_reviews_map
from features.predictive_pipeline.travel_optimizer import optimize_itinerary_sequence


5-Day Weather Forecast for Goa:
Day 1: Moderate rain, Max: 27.3°C, Min: 25.4°C, Rain Chance: 96%
Day 2: Patchy rain nearby, Max: 27.7°C, Min: 25.7°C, Rain Chance: 87%
Day 3: Patchy rain nearby, Max: 27.8°C, Min: 25.2°C, Rain Chance: 89%
Day 4: Patchy rain nearby, Max: 27.8°C, Min: 25.5°C, Rain Chance: 88%
Day 5: Moderate rain, Max: 26.5°C, Min: 25.1°C, Rain Chance: 87%


In [4]:
# ===============================
# 2️⃣ Base Itinerary Generation
# ===============================
parsed_input = {
    "location": "Jaipur",
    "duration_days": 2,
    "budget": 5000,
    "themes": ["heritage", "food"]
}

summary = "Visitors love Jaipur for its forts, palaces, and street food."

itinerary_data = generate_itinerary(parsed_input, summary)
itinerary_data


Client Created successfully
```json
{
  "itinerary": [
    {
      "day": 1,
      "morning": "Start early with a visit to Amer Fort (Amber Fort). Take a shared jeep up to the fort (around 50 INR per person) or walk. Explore the intricate palaces, courtyards, and Sheesh Mahal. Enjoy a hearty breakfast at a local eatery near the fort, perhaps a traditional Pyaaz Kachori and Chai (approx. 100 INR).",
      "afternoon": "Head back towards the city center. Visit Hawa Mahal (Palace of Winds) for its iconic facade (no need to go inside for budget/time). Then, walk through the vibrant Johari Bazaar, known for its jewelry and textiles, and indulge in some local street food like Samosa or Mirchi Bada (approx. 150 INR). Afterwards, explore Jantar Mantar (astronomical observatory, entrance 200 INR for Indians).",
      "evening": "Stroll through the bustling Bapu Bazaar, famous for its leather goods and mojari footwear. For dinner, experience local Rajasthani thali at a budget-friendly restaurant

{'itinerary': [{'day': 1,
   'morning': 'Start early with a visit to Amer Fort (Amber Fort). Take a shared jeep up to the fort (around 50 INR per person) or walk. Explore the intricate palaces, courtyards, and Sheesh Mahal. Enjoy a hearty breakfast at a local eatery near the fort, perhaps a traditional Pyaaz Kachori and Chai (approx. 100 INR).',
   'afternoon': 'Head back towards the city center. Visit Hawa Mahal (Palace of Winds) for its iconic facade (no need to go inside for budget/time). Then, walk through the vibrant Johari Bazaar, known for its jewelry and textiles, and indulge in some local street food like Samosa or Mirchi Bada (approx. 150 INR). Afterwards, explore Jantar Mantar (astronomical observatory, entrance 200 INR for Indians).',
   'evening': 'Stroll through the bustling Bapu Bazaar, famous for its leather goods and mojari footwear. For dinner, experience local Rajasthani thali at a budget-friendly restaurant like Thali House or Laxmi Mishtan Bhandar (approx. 300-400 

In [ ]:
reddit_comments = fetch_reddit_comments(parsed_input["location"])

print(f"Fetched {len(reddit_comments)} comments from Reddit.")

print(reddit_comments[:2])  # Display first 2 comments for brevity



Fetched 50 comments from Reddit.
[{'title': 'I traveled through India for 15 Days (New Delhi, Jaipur, Udaipur): my honest experience.', 'comments': ['I did the same route as you but had a terrible experience in Udaipur. I won\'t blame the city for my inconveniences, but I feel like Udaipur\'s "nice" walkable part is not as nice and walkable as people may think. And once you are out of that touristy oasis, It\'s a regular north Indian city. I wouldn\'t recommend it for a digital nomad but for a 2/3 day trip, It\'s cool. Those few streets where cars are not allowed are indeed an oasis in India.', 'Nice.\n\nI hear the Southern states are easier for beginners. Eg Kerela, Goa, Tamil Nadu etc.\n\nSmart move regarding hygeine. You chose wisely.', 'Glad you enjoyed it. I’m an Indian living abroad and I really enjoyed doing Udaipur -> Khumbalgarh -> Jodhpur when I was younger. \n\nWhere were you guys staying though? I feel like most of the drawbacks could be avoided by staying at any of the big

In [10]:
# reddit_comments is your fetched data
# Flatten all comment texts into a single list
comment_texts = []

for post in reddit_comments:
    # Ensure 'comments' exists and is a list
    if "comments" in post and isinstance(post["comments"], list):
        for c in post["comments"]:
            # Only include valid strings
            if isinstance(c, str) and not c.startswith("**Notice:**") and c.lower() != "[removed]":
                comment_texts.append(c)

# Now you can pass it to your local tips extractor
local_tips = extract_local_tips(comment_texts)


In [11]:
print(local_tips)

['I feel like most of the drawbacks could be avoided by staying at any of the big resorts in the area', 'India isn’t a bad country to visit, but definitely not a beginner country', 'I wouldn’t call it my favorite country because reality is that it can be hot, dirty, and inconvenient', 'I’m currently in Mumbai and am constantly loving it or ready to leave the country entirely', 'I am an individual and not a spam trying to sell something 😛', 'Your Agra report is the reason why I tell people to avoid Agra', 'There are times that my friend updates me about the seat and platform number because my ticket doesn’t include any information (Jaipur-Delhi route) \n\nYou can also book through makemytrip afaik but I can’t access the train option from my country', 'You can try Khamma Gani restaurant', 'Did you have a chance to try any of those, or did you stick more to the classic options', 'While I was inside the train, I was trying to book a hotel room because my flight was announced that it was de

In [14]:
# ===============================
# 4️⃣ Build Dynamic Place Reviews Map
# ===============================
comment_texts = []

for post in reddit_comments:
    # Ensure 'comments' exists and is a list
    if "comments" in post and isinstance(post["comments"], list):
        for c in post["comments"]:
            # Only include valid strings
            if isinstance(c, str) and not c.startswith("**Notice:**") and c.lower() != "[removed]":
                comment_texts.append(c)
                
place_reviews_map = build_place_reviews_map(comment_texts)

print("🗺 Place Reviews Map:")
print(place_reviews_map)


🗺 Place Reviews Map:
{'Udaipur': ['I did the same route as you but had a terrible experience in Udaipur', 'I won\'t blame the city for my inconveniences, but I feel like Udaipur\'s "nice" walkable part is not as nice and walkable as people may think', 'I’m an Indian living abroad and I really enjoyed doing Udaipur -> Khumbalgarh -> Jodhpur when I was younger', 'Skipped Udaipur and went up to Dharamshala instead', 'Udaipur is incredible – one of my favorite cities in India, and certainly one of the most beautiful', 'I will be going to these places plus Udaipur, Mumbai and i look forward to it', "If you're going to Jaipur you should also check out Udaipur", 'Udaipur is an absolute blast', 'I heard about Udaipur while I was traveling so  I definitely missed it', 'Anything you recommend to do and eat in Udaipur', 'q=india_tourism+Reddit+First-ever family trip in Nov 2025 - Jaipur vs Rishikesh+Mussoorie vs Udaipur vs Puri', 'If religios, go to Moti Dungri and birla temple, if possible add U

In [15]:
# ===============================
# 5️⃣ Crowd Optimization
# ===============================
itinerary_data = optimize_crowd(itinerary_data, place_reviews_map)

print("👥 Crowd Optimized Itinerary:")
print(itinerary_data)


👥 Crowd Optimized Itinerary:
{'itinerary': [{'day': 1, 'morning': 'Start early with a visit to Amer Fort (Amber Fort). Take a shared jeep up to the fort (around 50 INR per person) or walk. Explore the intricate palaces, courtyards, and Sheesh Mahal. Enjoy a hearty breakfast at a local eatery near the fort, perhaps a traditional Pyaaz Kachori and Chai (approx. 100 INR).', 'afternoon': 'Head back towards the city center. Visit Hawa Mahal (Palace of Winds) for its iconic facade (no need to go inside for budget/time). Then, walk through the vibrant Johari Bazaar, known for its jewelry and textiles, and indulge in some local street food like Samosa or Mirchi Bada (approx. 150 INR). Afterwards, explore Jantar Mantar (astronomical observatory, entrance 200 INR for Indians).', 'evening': 'Stroll through the bustling Bapu Bazaar, famous for its leather goods and mojari footwear. For dinner, experience local Rajasthani thali at a budget-friendly restaurant like Thali House or Laxmi Mishtan Bhand

In [16]:
# ===============================
# 6️⃣ Weather Optimization
# ===============================
itinerary_data = optimize_weather(itinerary_data, parsed_input["location"])

print("⛅ Weather Optimized Itinerary:")
print(itinerary_data)


⛅ Weather Optimized Itinerary:
{'itinerary': [{'day': 1, 'morning': 'Start early with a visit to Amer Fort (Amber Fort). Take a shared jeep up to the fort (around 50 INR per person) or walk. Explore the intricate palaces, courtyards, and Sheesh Mahal. Enjoy a hearty breakfast at a local eatery near the fort, perhaps a traditional Pyaaz Kachori and Chai (approx. 100 INR). (Great time for outdoor activities)', 'afternoon': 'Head back towards the city center. Visit Hawa Mahal (Palace of Winds) for its iconic facade (no need to go inside for budget/time). Then, walk through the vibrant Johari Bazaar, known for its jewelry and textiles, and indulge in some local street food like Samosa or Mirchi Bada (approx. 150 INR). Afterwards, explore Jantar Mantar (astronomical observatory, entrance 200 INR for Indians). (Great time for outdoor activities)', 'evening': 'Stroll through the bustling Bapu Bazaar, famous for its leather goods and mojari footwear. For dinner, experience local Rajasthani tha

In [23]:
# ===============================
# 7️⃣ Travel Optimization
# ===============================
import datetime
print("🚗 Travel Optimized Itinerary:")
# Start optimization from 9 AM today
optimized_itinerary = optimize_itinerary_sequence(itinerary_data, start_time=datetime.datetime.now().replace(hour=9, minute=0))

print(optimized_itinerary)
# itinerary_data = optimize_itinerary_sequence(itinerary_data)




🚗 Travel Optimized Itinerary:


TypeError: optimize_itinerary_sequence() got an unexpected keyword argument 'start_time'